# Wikipedia Notable Life Expectancies

# [Notebook 2 of 4: Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean_thanak_2022_06_13.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the wp_life_expect_raw_complete dataset
conn = sql.connect("wp_life_expect_raw_complete.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_raw_complete", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133900 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133898,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133899,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
43374,December 2009,9,Rodrigo Carazo Odio,", 82, Costa Rican politician, President (1978–1982), heart failure.",https://en.wikipedia.org/wiki/Rodrigo_Carazo_Odio,4
73781,January 2015,6,Sir John Mason,", 91, British meteorologist.",https://en.wikipedia.org/wiki/John_Mason_(meteorologist),26
128150,November 2021,1,Bruno Moretti,", 80, Australian athlete, Paralympic champion (1968).",https://en.wikipedia.org/wiki/Bruno_Moretti_(Paralympian),8
55564,May 2012,9,Constantin Piron,", 80, Belgian physicist.",https://en.wikipedia.org/wiki/Constantin_Piron,4
78387,September 2015,16,Bob Cleary,", 79, American ice hockey player, Olympic gold medalist (1960).",https://en.wikipedia.org/wiki/Bob_Cleary,4


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,900 rows and 6 columns.
- `month_year` contains the month and year of death, while `day` contains the day of the month of death.
- `name` is the notable person's name.  It is a nominal feature that will not be used for analysis, but will be maintained for any referencing needs.
- `info` contains multiple items including the notable person's "age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, (and) cause of death (if known)."
- `link` is the url to the notable person's individual Wikipedia page.  If such a page does not exist, there is either a non-working link (https://en.wikipedia.orgNone), or the link is to a page with a message that the page does not exist for that individual.  `link` is a unique identifier for all entries, except the 6 with the non-working link, which do have unique `name` values from each other.
- `num_references` contains the number of references on the notable person's individual Wikipedia page.  This feature serves as a proxy measure of notability.
- Prior to EDA, our task will be to extract the individual elements that are comined in `month_year` and `info` columns.

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

In [6]:
# Checking duplicate rows
df.duplicated().sum()

0

<IPython.core.display.Javascript object>

In [7]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000000
day               0.000000
name              0.004481
info              0.000000
link              0.000000
num_references    0.000000
dtype: float64

<IPython.core.display.Javascript object>

In [8]:
# Checking number of missing values per row (not necessary here, but done to keep process standard)
df.isnull().sum(axis=1).value_counts()

0    133894
1         6
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- Our dataset was saved to and read from the database without any hiccups.
- As expected, we have 6 entries that are missing `name`, but we will find it in their `info` values.
- All columns are currently of object type.  We will need to appropriately typecast them after separating the information in `month_year` and `info`.

## Data Cleaning

### Addressing Missing `name` Values

In [9]:
# Checking rows with missing name values
missing_name = df[df["name"].isna()]
missing_name

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- These rows vary from the main set as there is a substring containing the person's name at the start of the `info` string.
- As there are so few rows missing `name`, let us address this issue first.
- Within a for loop, we will split the `info` value into a list and extract `name` from the first list entry.  
- Then we will replace the name with an empty string within `info`.

In [10]:
# For loop to copy name value from info value and remove name from info value
treat_rows = missing_name.index
for i in treat_rows:
    info = df.loc[i, "info"]
    info_lst = info.split(sep=",", maxsplit=1)

    name = info_lst[0].strip()
    df.loc[i, "name"] = name
    df.loc[i, "info"] = re.sub(name, "", info).strip()

# Re-check rows
df.loc[treat_rows, :]

,month_year,day,name,info,link,num_references
18937,August 2001,11,Kevin Kowalcyk,", 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,0
24985,January 2004,22,Vincent Palmer,", 37, British criminal.",https://en.wikipedia.orgNone,0
27458,March 2005,1,Barry Stigler,", 57, American voice actor.",https://en.wikipedia.orgNone,0
34077,July 2007,11,Nana Gualdi,", 75, German singer and actress.",https://en.wikipedia.orgNone,0
64769,September 2013,29,Scott Workman,", 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,0
106613,September 2019,12,Thami Shobede,", 31, Singer Songwriter",https://en.wikipedia.orgNone,0


<IPython.core.display.Javascript object>

#### Observations:
- Missing `name` values have been addressed and those names have been removed from `info` values.

In [11]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133900 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  133900 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- We have no remaining missing values.
- Let us treat the `month_year` column next, by separating into two new columns `year` and `month`.

### Separating `month` and `year`

In [12]:
# Separating month and year into 2 columns and typecasting year as integer
df.loc[:, "year"] = df["month_year"].apply(lambda x: x.split(sep=" ")[1].strip())
df["year"] = df["year"].apply(lambda x: int(x))

df.loc[:, "month"] = df["month_year"].apply(lambda x: x.split(sep=" ")[0])
df.head(2)

,month_year,day,name,info,link,num_references,year,month
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January


<IPython.core.display.Javascript object>

In [13]:
# Dropping month_year column
df.drop("month_year", axis=1, inplace=True)
df.head(2)

,day,name,info,link,num_references,year,month
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January


<IPython.core.display.Javascript object>

### Treating `info`
First, we will define two functions to help identify rows that match a given regular expression pattern or list of patterns.  Then, we will start by examining the `info` column in a sample of the dataset.

#### Function to Save Indices of Rows Matching Regular Expressions Pattern to a List and Print Number of Rows with Match

In [14]:
# Define a function that takes dataframe, column name, and re pattern as arguments and returns list of indices
# for which column value matches re pattern
def rows_with_pattern(dataframe, column, pattern):
    """
    Takes input of dataframe, column name, and re pattern 
    and returns list of indices for rows that contain match
    for pattern anywhere within value for given column.
    
    dataframe: dataframe
    column: column name
    pattern: re pattern
    """
    index_list = []

    for i in dataframe.index:
        item = dataframe.loc[i, column]
        match = re.search(pattern, item)
        if match:
            index_list.append(i)
    print(
        f"There are {len(index_list)} rows with matching pattern in column '{column}'."
    )
    return index_list

<IPython.core.display.Javascript object>

#### Function to Use rows_with_pattern Function for Multiple Regular Expression Patterns

In [15]:
# Define a function that calls rows_with_pattern function for multiple re patterns
# returning a single list of indices for all rows with any pattern match


def multiple_patterns(dataframe, column, patterns):
    """
    Takes input dataframe, column, and list of re patterns and returns single list 
    of indices for rows in which a match for any pattern is found with re.search
    
    dataframe: dataframe
    column: column name
    patterns: list of re patterns
    """
    rows_combined = []

    # For loop to check each pattern
    for pattern in patterns:

        # List and number of rows matching each pattern
        print(pattern)
        rows_to_check = rows_with_pattern(dataframe, column, pattern)
        print("")

        # Add list for each pattern to combined list
        rows_combined += rows_to_check

    return rows_combined

<IPython.core.display.Javascript object>

#### Checking a Sample

In [16]:
# Checking a sample of info
df.sample(5)

,day,name,info,link,num_references,year,month
87551,22,Glenn D. Paige,", 87, American political scientist.",https://en.wikipedia.org/wiki/Glenn_D._Paige,26,2017,January
30628,11,Floyd Patterson,", 71, American former boxing heavyweight champion, Alzheimer's disease and prostate cancer.",https://en.wikipedia.org/wiki/Floyd_Patterson,43,2006,May
25070,6,Byron George Skelton,", 98, American judge (United States Court of Appeals for the Federal Circuit, United States Court of Claims).",https://en.wikipedia.org/wiki/Byron_George_Skelton,1,2004,February
10773,12,Lauri Kivekäs,", 94, Finnish businessman and politician.",https://en.wikipedia.org/wiki/Lauri_Kivek%C3%A4s,1,1998,February
75006,9,Villa Castelli helicopter collision,Notable French people killed in the :\n,https://en.wikipedia.org/wiki/Villa_Castelli_helicopter_collision,14,2015,March


<IPython.core.display.Javascript object>

#### Observations:
- We can see that `info` has variety, so will take some cleaning effort:
    - There are number values for years in addition to ages.
    - There is extra information inside of parentheses, that we likely don't need.
    - Some entries lack cause of death.
    - Some entries have multiple roles listed, with separating commas.
    - There are nationalities with multiple words.
    - `info` contains capital letters that are not part of citizenship.
- A strategic approach to cleaning the `info` column is needed, as follows:
    1. Remove all of the 4-digit year information, as we do not need it.
    2. Drop all entries that are lacking digits, as they are missing the  target `age` information.
    2. Extract any parentheses and their contents from `info` to new column `info_parenth`, as it likely won't be needed but we will preserve it, for now.

#### Removing 4-digit Years from `info`

In [18]:
# Pattern for re
pattern = r"\d{4}"

# Finding indices of rows that have pattern
rows_to_check = rows_with_pattern(df, "info", pattern)

# Checking a sample of rows
df.loc[rows_to_check, :].sample(2)

There are 24899 rows with matching pattern in column 'info'.


,day,name,info,link,num_references,year,month
72807,18,Ahmad Lozi,", 89, Jordanian politician, Prime Minister (1971–1973), President of the Senate (1984–1997).",https://en.wikipedia.org/wiki/Ahmad_Lozi,15,2014,November
32490,31,Liese Prokop,", 65, Austrian athlete and Minister of the Interior (2004–2006), aortic dissection.",https://en.wikipedia.org/wiki/Liese_Prokop,6,2006,December


<IPython.core.display.Javascript object>

In [20]:
# For loop to replace year with empty string
for i, item in enumerate(df["info"]):
    match = re.search(pattern, item)
    if match:
        df.loc[i, "info"] = re.sub(pattern, "", df.loc[i, "info"])

# Rechecking number and example rows after treatment
rows_to_check = rows_with_pattern(df, "info", pattern)
pd.concat([df[df["name"] == "Ahmad Lozi"], df[df["name"] == "Liese Prokop"]])

There are 0 rows with matching pattern in column 'info'.


,day,name,info,link,num_references,year,month
72807,18,Ahmad Lozi,", 89, Jordanian politician, Prime Minister (–), President of the Senate (–).",https://en.wikipedia.org/wiki/Ahmad_Lozi,15,2014,November
32490,31,Liese Prokop,", 65, Austrian athlete and Minister of the Interior (–), aortic dissection.",https://en.wikipedia.org/wiki/Liese_Prokop,6,2006,December


<IPython.core.display.Javascript object>

#### Observations:
- The 4-digit years have been removed.
- Our next step is to check for any rows that lack digits and drop them.
- We will do this by overwriting the dataframe with only the rows that do contain digits.

#### Checking and Dropping Rows Lacking Digits (and therefore Age Data) within `info`

In [23]:
# Pattern for re
pattern = r"\d"

# Finding indices of rows that have pattern
has_digits = rows_with_pattern(df, "info", pattern)
print(
    f"\nThere are {len(df) - len(has_digits)} rows without numbers in the info column."
)

# Dropping rows missing age data, resetting index, and checking new shape of df
df = df.loc[has_digits, :]
df.reset_index(inplace=True, drop=True)
df.shape

There are 132830 rows with matching pattern in column 'info'.

There are 1070 rows without numbers in the info column.


(132830, 7)

<IPython.core.display.Javascript object>

#### Observations:
- 1070 rows were removed as they lacked any digits and, therefore, the target data for `age`.
- Next, we will extract parentheses and their contents from `info` to a new column `info_parenth`.

#### Removing Information within Parentheses from `info` and saving to new column `info_parenth`

In [24]:
# Regular expression for parenthesis and its contents
pattern = r"\(.*\)"

# Finding indices of rows that have pattern
rows_to_check = rows_with_pattern(df, "info", pattern)

# Checking a sample of rows
df.loc[rows_to_check, :].sample(2)

There are 49884 rows with matching pattern in column 'info'.


,day,name,info,link,num_references,year,month
76053,11,David Premack,", 89, American psychologist (Premack's principle).",https://en.wikipedia.org/wiki/David_Premack,20,2015,June
115808,15,P. Vetrivel,", 60, Indian politician, Tamil Nadu MLA (–), septic shock from COVID-19.",https://en.wikipedia.org/wiki/P._Vetrivel,4,2020,October


<IPython.core.display.Javascript object>

In [25]:
# For loop to extract parenthesis and its contents from info to info_parenth
for i, item in enumerate(df["info"]):
    match = re.search(pattern, item)
    if match:
        df.loc[i, "info_parenth"] = match.group(0)
        df.loc[i, "info"] = re.sub(pattern, "", df.loc[i, "info"])

# Rechecking number and example rows after treatment
rows_to_check = rows_with_pattern(df, "info", pattern)
pd.concat([df[df["name"] == "David Premack"], df[df["name"] == "P. Vetrivel"]])

There are 0 rows with matching pattern in column 'info'.


,day,name,info,link,num_references,year,month,info_parenth
76053,11,David Premack,", 89, American psychologist .",https://en.wikipedia.org/wiki/David_Premack,20,2015,June,(Premack's principle)
115808,15,P. Vetrivel,", 60, Indian politician, Tamil Nadu MLA , septic shock from COVID-19.",https://en.wikipedia.org/wiki/P._Vetrivel,4,2020,October,(–)


<IPython.core.display.Javascript object>

#### Observation:
- Parentheses and information within has been removed from `info` and assigned to `info_parenth`.
- Next, we will iterate through the rows, splitting `info` on commas and assigining the respective list values to new individual columns `info_0`, `info_`, etc.  
- Though we can keep in mind the Wikipedia-defined fields, we will take the general approach of treating column by column, after splitting `info`, varying as indicated to obtain specific feature information.

#### Splitting `info` on Commas into Separate Columns

In [26]:
# For loop to split info on commas and separate into respective new columns and removing leading/trailing white space and periods
for i, item in enumerate(df["info"]):
    info_lst = item.split(",")

    for j in range(len(info_lst)):
        df.loc[i, f"info_{j}"] = info_lst[j].strip(" .")

# Checking the first 2 rows
df.head(2)

,day,name,info,link,num_references,year,month,info_parenth,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,NaN,,86,British dancer,ballet designer and director,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,NaN,,68,Irish economist,writer,and academic,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [27]:
# Checking the last 2 rows
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_0,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11
132828,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,(),,69,Russian volleyball player,Olympic champion and coach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132829,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,NaN,,86,Chinese engineer,member of the Chinese Academy of Engineering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

#### Observations:
- The `info` value is successfully divided and we can proceed through it column by column.
- We will check the set of values for the first two columns, for age.

### `info_0`

In [ ]:
# Checking unique value counts
df["info_0"].value_counts()

#### Observations:
- The vast majority of rows have an empty string for this field.
- There is one row representing a group, rather than an individual, and we will drop it.
- We should verify the name and age information for the remainder of unique values in `info_0`.

#### Dropping Entry for Group

In [ ]:
# Checking the entry representing a group
group_entry = df[
    df["info_0"]
    == "Notable ice hockey players and coaches among the 44 killed in the :\n"
]
group_entry

In [ ]:
# Dropping group entry, resetting index, and checking new shape of df
df.drop(group_entry.index, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

#### Examining Rows with Atypical `info_0` Values

In [ ]:
# Examining rows with atypical info_0 values
list_to_check = df["info_0"].value_counts().index.to_list()

verify_df = pd.DataFrame()
for item in list_to_check[1:]:
    verify_df = pd.concat([verify_df, df[df["info_0"] == item]])
verify_df

#### Observations:
- The majority of rows contain additional aliases or titles within `info_0`, that we don't need, but we can leave in place for now.  
- There are a few rows that will need to be treated individually to correct the name value, as follows:
    1. Entry is for Mike Alexander whose band was Evile.
    2. Entry is for Herbert Wiere who performed slapstick.
    3. Entry is for Sarah-Jayne Mulvihill who was a Flight Lieutenant.
    4. Entry is for Douglass Scott who was killed by Demetreus Nix.
    5. Entry is for Kim Hwan-Sung who was a member of the band NRG.
- We can replace the `name` value with the `info_0` value for these rows as well as proceed with hard-coding the correct values for info_2 and info_3 fields to match the Wikipedia pattern, but staying true to the information scraped.
- The row with "Nearly 3" value for `info_0` represents a group, rather than an individual, so will be dropped, after treating the above rows.
- We can proceed to extract age from `info_0` for the few rows that contain it here instead of in `info_1`.

#### Treating 5 rows with Name in `info_0`

In [ ]:
# List of names values in info_0
values_lst = [
    "Mike Alexander",
    "Herbert Wiere",
    "Sarah-Jayne Mulvihill",
    "Douglas Scott",
    "Kim Hwan-Sung",
]

In [ ]:
# For loop to copy name from info_0 to name
for i in df[df["info_0"].isin(values_lst)].index.to_list():
    df.loc[i, "name"] = df.loc[i, "info_0"]

# Hard-coding info_2 and info_3 values for Kim Hwan-Sung
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/NRG_(South_Korean_band)"
].index.to_list()
df.loc[index, "info_2"] = "South Korean musician"

df.loc[index, "info_3"] = "respiratory illness"

# # Hard-coding info_2 and info_3 values for Douglass Scott
index = df[
    df["link"]
    == "https://en.wikipedia.org/w/index.php?title=Demetreus_Nix&action=edit&redlink=1"
].index.to_list()
df.loc[index, "info_2"] = "student"

df.loc[index, "info_3"] = "murdered"

# # Hard-coding info_2 and info_3 values for Sarah-Jayne Mulvihill
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/Flight_Lieutenant"
].index.to_list()
df.loc[index, "info_2"] = "British servicewoman"

df.loc[index, "info_3"] = "killed in action"

In [ ]:
# Rechecking updated rows
df[df["info_0"].isin(values_lst)]

#### Dropping Entry for Group

In [ ]:
# Checking the entry representing a group
group_entry = df[df["info_0"] == "Nearly 3"]
group_entry

In [ ]:
# Dropping group entry, resetting index, and checking new shape of df
df.drop(group_entry.index, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

#### Extracting `age` from `info_0`

In [ ]:
# Pattern for re
pattern = r"(\d{1,3})"

# Checking rows with pattern
rows_to_check = rows_with_pattern(df, "info_0", pattern)

In [ ]:
# For loop to extract age from info_0 to age
for i, item in enumerate(df["info_0"]):
    match = re.search(pattern, item)
    if match:
        df.loc[i, "age"] = int(match.group(1))
        df.loc[i, "info_0"] = re.sub(pattern, "", df.loc[i, "info_0"])

# Re-checking info_0 and age for pattern
rows_to_check = rows_with_pattern(df, "info_0", pattern)
df[df["age"].notna()]

#### Observations:
- The new `age` column has been added successfully.
- We are finished processing `info_0`.

#### Re-checking Unique Values for `info_0` and Dropping the Column

In [ ]:
# Re-checking unique values for info_0 prior to dropping it
df["info_0"].unique()

In [ ]:
# Dropping info_0
df.drop("info_0", axis=1, inplace=True)

#### Observations:
- We are ready to move on to processing `info_1`, which should primarily consist of age values, per the defined Wikipedia fields.

### `info_1`

#### Unique Values

In [ ]:
# Checking unique values
df["info_1"].unique()

#### Observations:
- There is a lot of variety in the format of the age data.
- Also, this field contains several values that we would expect in info_2 and beyond.
- Let us take the approach of extracting age values first.

#### Examining Unique Formats for Age Data

In [ ]:
# Pattern for re
pattern = r"\d"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking unique values for column
df.loc[rows_to_check, :]["info_1"].unique()

#### Observations:
- The data for age within `info_1` is in the following formats:  
    - single integer ("age", "age ", "age.")
    - range of 2 integers (separators '-', '–', '/', and ' or ')
    - range of 2 integers with only unit value for second number ('age1/age2-2nd-digit')
    - age in days or months ('age days', 'age-months')
    - estimates ('c. age', 'c.age',  'age?', 'ages' (e.g. 80s), "age+"
- There are some specific rows that need to be examined, with the following values for `info_1`:
    - 1995
    - 1996
    - 1997
    - German Olympic sailor [1]
    - Taiwanese failed assassin in the 3-19 shooting incident
    - 255
    - 176
    - the first wild bear in Germany in 170 years
    - c. 3500
    - common chimpanzee 55
    - Maltese 15
    - c.1000
    - Tree of the Year 150
- We will need to be strategic in the order in which we extract age from `info_1`.
- First, we will look at the atypical values listed above.

#### Examining Rows with Digits and Atypical Values for `info_2`

In [ ]:
# List of atypical info_1 values for rows with digits
values_lst = [
    "1995",
    "1996",
    "1997",
    "German Olympic sailor [1]",
    "Taiwanese failed assassin in the 3-19 shooting incident",
    "255",
    "176",
    "the first wild bear in Germany in 170 years",
    "c. 3500",
    "common chimpanzee 55",
    "Maltese 15",
    "c.1000",
    "Tree of the Year 150",
]

df[df["info_1"].isin(values_lst)]

#### Observations:
- Age data is either missing or the entry is for a member of a non-human species.
- We will drop all of these rows.

#### Dropping Rows for Non-Human Entries or Entries Missing Age Data

In [ ]:
# List of indexes to be dropped
drop_rows = df[df["info_1"].isin(values_lst)].index.to_list()

# Dropping rows, resetting index, and checking new shape of df
df.drop(drop_rows, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

#### Observations:
- With those rows addressed, we will begin extracting age in days and months and convert them to years.

### Extracting `age` from `info_1`

#### Step 1: Age in Years and Months

In [ ]:
# Dictionary of patterns for days and months formats as keys and factor to convert to years
patterns = {
    r"(\d{1,3})( days)": 365,
    r"(\d{1,3})(-months)": 12,
    r"(\d{1,3})( months)": 12,
}

# List and number of rows matching patterns
rows_to_check = multiple_patterns(df, "info_1", patterns)

In [ ]:
# For loop to extract age in days and months fromm info_1 and convert to years and save in age
for key, value in patterns.items():
    for i, item in enumerate(df["info_1"]):
        match = re.search(key, item)
        if match:
            age = int(match.group(1)) / value
            df.loc[i, "age"] = age
            df.loc[i, "info_1"] = re.sub(key, "", df.loc[i, "info_1"])

# Re-check number of rows matching patterns
rows_to_check = multiple_patterns(df, "info_1", patterns)

# Checking updated rows
df[df["age"].notna()]

#### Observations:
- We have successfully captured the age in days and months values and converted them to years.
- The other rows are in place already from our treatment of `info_0`.
- Next, we will address entries that contain two age values as a range, starting first with those that have a single digit as the second number.

#### Step 2: Extracting `age` from `info_1` for Entries with Age Estimate Containing Age Range with 2 Values

#### Ranges with Single Digit as Upper-end

In [ ]:
# Pattern for re
pattern = r"(\d{1,3})(/)(\d)\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking sample of rows
df.loc[rows_to_check, :].sample(2)

In [ ]:
# For loop to find rows with values and pattern and calculate and extract age to age column and remove age from info_1
for i in df[df["age"].isna()].index:
    item = df.loc[i, "info_1"]
    match = re.search(pattern, item)
    if match:
        age_1 = int(match.group(1))
        age_2 = int(match.group(3))
        units = ((age_1 % 10) + age_2) / 2
        tens = age_1 - (age_1 % 10)
        age = tens + units
        df.loc[i, "age"] = age
        df.loc[i, "info_1"] = re.sub(pattern, "", df.loc[i, "info_1"])

# Checking example rows
pd.concat([df[df["name"] == "Sayed Khalifa"], df[df["name"] == "Joji Banuve"]])

#### Other Ranges with Two Values

In [ ]:
# Pattern for re
pattern = r"(\d{1,3})(-|–|/| or )(\d{1,3})"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking sample of rows
df.loc[rows_to_check, :].sample(2)

In [ ]:
# For loop to find rows with values and pattern and calculate and extract age to age column and remove age from info_1
for i in df[df["age"].isna()].index:
    item = df.loc[i, "info_1"]
    match = re.search(pattern, item)
    if match:
        age = (int(match.group(1)) + int(match.group(3))) / 2
        df.loc[i, "age"] = age
        df.loc[i, "info_1"] = re.sub(pattern, "", df.loc[i, "info_1"])

# Checking example rows
pd.concat(
    [df[df["name"] == "Paulo de Tarso Alvim"], df[df["name"] == "Ahmed Al Khattab"]]
)

#### Observations:
- Next, we will extract from the entries with straightforward single integer age values, including the formats: "age", "age ", 'age.'.
- More vague estimates are excluded here to allow closer examination, as they are more likely to by atypical entries.

#### Step 3: Age as Single Integer (Excluding Estimates)

In [ ]:
# List of patterns for age formats with single integer for age
patterns = [r"^(\d{1,3})$", r"^(\d{1,3})\s", r"^(\d{1,3})\.\s"]

# List and number of rows matching patterns
rows_to_check = multiple_patterns(df, "info_1", patterns)

In [ ]:
# For loop to check age pattern in info_1, save age to age column, and remove from age from info_1
for i, item in enumerate(df["info_1"]):
    for pattern in patterns:
        match = re.search(pattern, item)
        if match:
            age = int(match.group(1))
            df.loc[i, "age"] = age
            df.loc[i, "info_1"] = re.sub(pattern, "", df.loc[i, "info_1"])

# Re-checking number of rows matching patterns
rows_to_check = multiple_patterns(df, "info_1", patterns)

# Checking first 2 rows
df.head(2)

In [ ]:
# Checking last 2 rows
df.tail(2)

In [ ]:
# Checking the number of remaining missing values for `age`
print(f'There are {df["age"].isna().sum()} remaining missing values for age.')

#### Observations:
- The rows with single integer age data have been addressed.
- There are only 294 remaining missing values for `age` after extracting age in days, months, single integer years, and 2-integer year range values.
- Let us check the rows containing 'c.', '+', or '?' in the age information.  We will do the ranges ending in 's', such as 80s, separately.

#### Entries with Age Data Containing 'c.', '+', or '?' for Estimate

In [ ]:
# Pattern for re
pattern = r"(c\.|\+|\?)"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Inspecting rows containing values
df.loc[rows_to_check, :]

#### Observations:
- Most of the entries are for people, but there are also one or more entries for the following:
    - carp
    - racehorse
    - chimpanzee
    - flamingo
    - cat
    - turkey
- We will proceed to check `info_2` for these values and drop these and other rows representing members of these other species.

#### Checking for Cat, Racehorse, Chimpanzee, Carp, and Flamingo in `info_2`

In [ ]:
# pattern for re
pattern = r"\b(cat|racehorse|chimpanzee|carp|flamingo|turkey)\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df[df["info_2"].notna()], "info_2", pattern)

#### Observations:
- There are sufficient rows to warrant checking species by species.

#### Cat Entries per `info_2`

In [ ]:
# Pattern for re
pattern = r"\b(cat)\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df[df["info_2"].notna()], "info_2", pattern)

# Inpsecting rows with pattern
df.loc[rows_to_check, :]

#### Observations:
- There is one person represented.  
- We can proceed to drop the others, using "cat", "cat of", and "cat in" patterns.

#### Dropping Entries for Cats per `info_2`

In [ ]:
# List of re patterns to find
patterns = [r"\bcat$", r"\b(cat of|cat in)\b"]

# List and number of rows matching pattern
rows_to_drop = multiple_patterns(df[df["info_2"].notna()], "info_2", patterns)

In [ ]:
# Dropping rows, resetting index, and checking new shape of df
df.drop(rows_to_drop, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

####  Racehorse Entries per `info_2`

In [ ]:
# Pattern for re
pattern = r"\bracehorse\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df[df["info_2"].notna()], "info_2", pattern)

# Inpsecting rows with pattern
df.loc[rows_to_check, :].sample(10)

#### Observations:
- There are several entries for people involved in the racehorse business.
- Values that end in 'racehorse' and 'racehorse and sire' can be removed.

#### Dropping Entries for Racehorses per `info_2`

In [ ]:
# List of re patterns to find
patterns = [r"\bracehorse$", r"\b(racehorse and sire)$"]

# List and number of rows matching pattern
rows_to_drop = multiple_patterns(df[df["info_2"].notna()], "info_2", patterns)

In [ ]:
# Dropping rows, resetting index, and checking new shape of df
df.drop(rows_to_drop, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

####  Chimpanzee, Flamingo,  Carp, and Turkey Entries per `info_2`

In [ ]:
# Defining pattern for re
pattern = r"\b(chimpanzee|flamingo|carp|turkey)\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df[df["info_2"].notna()], "info_2", pattern)

# Inpsecting rows with pattern
df.loc[rows_to_check, :]

#### Observations:
- All of these rows are for animals, so we will remove them.

#### Dropping Entries for Chimpanzees, Flamingos,  Carps, and Turkeys per `info_2`

In [ ]:
# Dropping rows, resetting index, and checking new shape of df
df.drop(rows_to_check, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

#### Observations:
- With those non-human entries addressed, we can return to processing `info_1`.
- Let us address the remaining entries with '?', '+', or 'c.', accepting the estimated age as `age`.
- We will treat the age estimates ending in 's' similarly, but separately.

#### Extracting `age` from `info_1` for Entries with Age Estimate Containing '?', '+', or 'c.'

In [ ]:
# Pattern for re
pattern = r"(c\.|\+|\?)"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking a sample of rows
df.loc[rows_to_check, :].sample(2)

In [ ]:
# List to identify rows
values = ["c.", "+", "?"]

# Pattern for re
pattern = r"\b(\d{1,3})\b"

# For loop to find rows with values and pattern and extract age to age column and remove age from info_1
for i in df[df["age"].isna()].index:
    item = df.loc[i, "info_1"]

    if any(value in item for value in values):
        match = re.search(pattern, item)

        if match:
            age = int(match.group(1))
            df.loc[i, "age"] = age
            df.loc[i, "info_1"] = re.sub(pattern, "", df.loc[i, "info_1"])

        for value in values:
            df.loc[i, "info_1"] = df.loc[i, "info_1"].replace(value, "")


# Re-checking number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking example rows
pd.concat([df[df["name"] == "Ataa Oko"], df[df["name"] == "Bob Randall"]])

#### Observations:
- Here, we see an example of two people with the same name, but the extraction of `age` from `info_1` was successful.
- The ages ending in 's' should be the only ones remaining in the `info_1` column.
- We will examine those now.

#### Extracting `age` from `info_1` for Entries with Age Estimate ending in 's'

In [ ]:
# Defining pattern for re
pattern = r"\b(\d{1,3})s\b"

# List and number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Inpsecting rows with pattern
df.loc[rows_to_check, :]

#### Observations:
- They are all for people, so we can extract `age` from `info` and set it to the middle value of the range.

In [ ]:
# For loop to find rows with values and pattern and extract age to age column and remove age from info_1
for i in df[df["age"].isna()].index:
    item = df.loc[i, "info_1"]
    match = re.search(pattern, item)
    if match:
        age = int(match.group(1))
        df.loc[i, "age"] = age + 5
        df.loc[i, "info_1"] = re.sub(pattern, "", df.loc[i, "info_1"])
    if "early " in item:
        df.loc[i, "info_1"] = df.loc[i, "info_1"].replace("early ", "")

# Re-checking number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking example rows
pd.concat([df[df["name"] == "Mary Dann"], df[df["name"] == "Timothy Apiyo"]])

### Checking for Any Missed Digits in `info_1` and for Remaining Missing Values for `age`

In [ ]:
# Pattern for re
pattern = r"\d"

# Re-checking number of rows matching pattern
rows_to_check = rows_with_pattern(df, "info_1", pattern)

# Checking number of missing values for age
print(f'\nThere are {df["age"].isna().sum()} remaining missing values for age.')

### Observations:
- All of the age data that had been in `info_1` has been successfully extracted.
- There are 218 remaining missing values for `age` that we hope to find in the other info columns.
- We will include the remaining values in `info_1` when we extract citizenship and role information.
- It is time to export the current dataframe to a SQLite database.

### Exporting Dataset to SQLite Database [wp_life_expect_clean1.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_clean1.db)

In [ ]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_life_expect_clean1.db")
df.to_sql("wp_life_expect_clean1", conn, index=False)

# [Proceed to Notebook 3 of  4:  Data Cleaning Part 2](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean2_thanak_2022_06_17.ipynb)